In [131]:
import json
import boto3
import requests
import datetime
from pytz import timezone
import numpy as np

def get_schedule(adjusted_time):
    def get_date(game, adjusted_time):
        AT = adjusted_time
        print(game['date'])
        start_time = convert_my_iso_8601(game['date'],timezone('EST'))
        return start_time - datetime.timedelta(hours=AT)

    def convert_my_iso_8601(iso_8601, tz_info):
        assert iso_8601[-1] == 'Z'
        iso_8601 = iso_8601[:-1] + '000'
        iso_8601_dt = datetime.datetime.strptime(iso_8601, '%Y-%m-%dT%H:%M:%S%f')
        return iso_8601_dt.replace(tzinfo=timezone('UTC'))#.astimezone(tz_info)
    
    def set_cron(date):
        minute = str(date.minute)
        hour = str(date.hour)
        dayofmonth = str(date.day)
        month = str(date.month)
        dayofweek = '?'
        year = str(date.year)
        return {"cron": 'cron({} {} {} {} {} {})'.format(minute, hour, dayofmonth, month, dayofweek, year),
                "name": '{}_{}_{}_{}{}'.format(month, dayofmonth, year, hour, minute).replace(',','.')}
        
    start = datetime.date.today()
    end = start + datetime.timedelta(days=7)
    start = start.strftime("%Y%m%d")
    end = end.strftime("%Y%m%d")

    url = "https://site.api.espn.com/apis/fantasy/v2/games/ffl/games?useMap=true&dates={}-{}&pbpOnly=true".format(start, end)
    r = requests.get(url)
    data = r.json()
    dates = list(np.unique([get_date(game, adjusted_time) for game in data['events']]))
    
    return [set_cron(date) for date in dates]

dates = get_schedule(0.5)

2019-12-01T18:00:00Z
2019-12-01T18:00:00Z
2019-12-01T18:00:00Z
2019-12-01T18:00:00Z
2019-12-01T18:00:00Z
2019-12-01T18:00:00Z
2019-12-01T18:00:00Z
2019-12-01T18:00:00Z
2019-12-01T21:05:00Z
2019-12-01T21:25:00Z
2019-12-01T21:25:00Z
2019-12-02T01:20:00Z
2019-12-03T01:15:00Z
2019-12-06T01:20:00Z


In [135]:
dates

[{'cron': 'cron(30 17 1 12 ? 2019)', 'name': '12_1_2019_1730'},
 {'cron': 'cron(35 20 1 12 ? 2019)', 'name': '12_1_2019_2035'},
 {'cron': 'cron(55 20 1 12 ? 2019)', 'name': '12_1_2019_2055'},
 {'cron': 'cron(50 0 2 12 ? 2019)', 'name': '12_2_2019_050'},
 {'cron': 'cron(45 0 3 12 ? 2019)', 'name': '12_3_2019_045'},
 {'cron': 'cron(50 0 6 12 ? 2019)', 'name': '12_6_2019_050'}]

In [133]:
cloudwatch_events = boto3.client('events')
print(d['name'])
roleArn = 'arn:aws:iam::534552671502:role/service-role/db_test-role-l88wdlrt'
arn = 'arn:aws:lambda:us-east-1:534552671502:function:update_espn_roster'
response = cloudwatch_events.put_rule(
    Name=d['name'],
    ScheduleExpression=d['cron'],
    State='ENABLED',
    Description='test description',
    RoleArn=roleArn
)

print(response)

12_1_2019_1730
{'ResponseMetadata': {'HTTPHeaders': {'x-amzn-requestid': '3a7f02c2-c35b-40bf-8f97-739fa3a9662a', 'content-type': 'application/x-amz-json-1.1', 'date': 'Fri, 29 Nov 2019 05:37:32 GMT', 'content-length': '71'}, 'RequestId': '3a7f02c2-c35b-40bf-8f97-739fa3a9662a', 'RetryAttempts': 0, 'HTTPStatusCode': 200}, 'RuleArn': 'arn:aws:events:us-east-1:534552671502:rule/12_1_2019_1730'}


In [134]:
arn = 'arn:aws:lambda:us-east-1:534552671502:function:update_espn_roster'
response = cloudwatch_events.put_targets(
    Rule=d['name'],
    Targets=[
        {
            'Arn': arn,
            'Id': 'default'
        }
    ]
)
print(response)

{'FailedEntryCount': 0, 'FailedEntries': [], 'ResponseMetadata': {'HTTPHeaders': {'x-amzn-requestid': 'f682148a-dac1-42b6-abda-bed10433c83d', 'content-type': 'application/x-amz-json-1.1', 'date': 'Fri, 29 Nov 2019 05:37:33 GMT', 'content-length': '41'}, 'RequestId': 'f682148a-dac1-42b6-abda-bed10433c83d', 'RetryAttempts': 0, 'HTTPStatusCode': 200}}


In [124]:
d['name']

'11_28_2019_170'

In [127]:
response = cloudwatch_events.remove_targets(
    Rule='11_28_2019_120',
    Ids=[
        'default',
    ]
)
print(response)

In [130]:
response = cloudwatch_events.delete_rule(
    Name='11_28_2019_120'
)
print(response)

{'ResponseMetadata': {'HTTPHeaders': {'x-amzn-requestid': 'dd082bd3-8c02-47a1-81d9-58e6d3d0ffc1', 'content-type': 'application/x-amz-json-1.1', 'date': 'Fri, 29 Nov 2019 05:12:17 GMT', 'content-length': '0'}, 'RequestId': 'dd082bd3-8c02-47a1-81d9-58e6d3d0ffc1', 'RetryAttempts': 0, 'HTTPStatusCode': 200}}


['cron(0 12 28 11 ? 2019)',
 'cron(0 16 28 11 ? 2019)',
 'cron(50 19 28 11 ? 2019)',
 'cron(30 12 1 12 ? 2019)',
 'cron(35 15 1 12 ? 2019)',
 'cron(55 15 1 12 ? 2019)',
 'cron(50 19 1 12 ? 2019)',
 'cron(45 19 2 12 ? 2019)']

In [71]:
dates = [{
    "competitors": game['competitors'],
    "date": game['date'],
    "odds": game['odds'],
    "scoringPlays": game['scoringPlays'],
    "isNational": game['broadcasts'][0]['isNational'],
} for game in data['events']]

[datetime.datetime(2019, 11, 28, 12, 30, tzinfo=<StaticTzInfo 'EST'>),
 datetime.datetime(2019, 11, 28, 16, 30, tzinfo=<StaticTzInfo 'EST'>),
 datetime.datetime(2019, 11, 28, 20, 20, tzinfo=<StaticTzInfo 'EST'>),
 datetime.datetime(2019, 12, 1, 13, 0, tzinfo=<StaticTzInfo 'EST'>),
 datetime.datetime(2019, 12, 1, 16, 5, tzinfo=<StaticTzInfo 'EST'>),
 datetime.datetime(2019, 12, 1, 16, 25, tzinfo=<StaticTzInfo 'EST'>),
 datetime.datetime(2019, 12, 1, 20, 20, tzinfo=<StaticTzInfo 'EST'>),
 datetime.datetime(2019, 12, 2, 20, 15, tzinfo=<StaticTzInfo 'EST'>)]

In [119]:
from ff_espn_api import League
import requests
import pandas as pd
from ff_espn_api.constant import *
import json


class Manager(League):
    def __init__(self, league_id, year, username, password):
        super().__init__(league_id, year, username=username, password=password)
        self.team = self.get_user_team()

    def get_lineup_slot_counts(self):
        url = self.ENDPOINT
        payload = {"view": "mSettings"}
        response = requests.get(url, params=payload, cookies=self.cookies)
        if response.ok is False:
            print("### ERROR: response error with get_roster function ###")
            print(json.dumps(response.json, indent=6))
        data = response.json() if self.year > 2018 else response.json()[0]
        lineup_slot_counts = data['settings']['rosterSettings']['lineupSlotCounts']
        return lineup_slot_counts

    def get_roster_data(self):
        roster_data = [{
            "id": player.playerId,
            "name": player.name,
            "projection": player.projected_points,
            "position": player.position,
            "eligible_slots": player.eligibleSlots,
            "current_slot": player.currentSlot,
            "lineup_locked": player.lineup_locked
        } for player in self.team.roster]
        return pd.DataFrame(roster_data)

    def get_adjusted_roster(self):
        roster = self.get_roster_data()
        roster['current_slot_id'] = [pos for r in roster.current_slot for pos in POSITION_MAP if POSITION_MAP[pos] == r]
        roster = roster.sort_values(['projection'], ascending=False).reset_index()
        lineup_slot_counts = self.get_lineup_slot_counts()
        for index in range(len(roster)):
            if roster.lineup_locked[index]:
                lineup_slot_counts[ str(roster.current_slot_id[index]) ] += -1

        adjusted_slot = []
        for index in range(len(roster)):
            position = roster.position[index]
            eligible_slots = roster.eligible_slots[index]
            current_slot_id = roster.current_slot_id[index]
            locked = roster.lineup_locked[index]
            print(roster.name[index])
            if (current_slot_id == 21) or locked:
                print(1)
                adjusted_slot_value = int(current_slot_id)
            elif lineup_slot_counts[ str(POSITION_MAP[position]) ] > 0:
                print(2)
                adjusted_slot_value = int(POSITION_MAP[position])
            else:
                slot_found = None
                for slot in lineup_slot_counts:
                    slot_num = int(slot)
                    if slot_num == 4:
                        print(lineup_slot_counts[slot])
                        print(POSITION_MAP[slot_num] in eligible_slots)
                    if (slot_num < 20) and (lineup_slot_counts[slot] > 0) and (POSITION_MAP[slot_num] in eligible_slots):
                        print(3)
                        slot_found = slot_num
                        break
                if slot_found is None:
                    print(4)
                    adjusted_slot_value = int(20)
                else:
                    print(5)
                    adjusted_slot_value = int(slot_found)
            lineup_slot_counts[str(adjusted_slot_value)] += -1
            adjusted_slot.append(adjusted_slot_value)

        roster['adjusted_slot'] = adjusted_slot
        return roster

    def make_lineup_adjustment(self):
        roster = self.get_adjusted_roster()
        team_adjustments = roster[roster.current_slot_id != roster.adjusted_slot]
        adjustments_needed = len(team_adjustments)
        if adjustments_needed > 0:
            adjustment = {
                "isLeagueManager": False,
                "teamId": "{}".format(self.team.team_id),
                "type": "ROSTER",
                "memberId": "{}".format(self.cookies['swid']),
                "scoringPeriodId": "{}".format(self.current_week),
                "executionType": "EXECUTE",
                "items": []
            }

            current_starters = team_adjustments[team_adjustments.current_slot_id < 20]
            if len(current_starters) > 0:
                move_starter_players = [{
                        "playerId": "{}".format(player[0]),
                        "type": "LINEUP",
                        "fromLineupSlotId": "{}".format(player[1]),
                        "toLineupSlotId": "{}".format(20)
                    } for player in current_starters[['id', 'current_slot_id']].values]
                adjustment["items"] = move_starter_players
                response = requests.post(self.ENDPOINT + "/transactions/", json=adjustment, cookies=self.cookies)
                if response.ok is False:
                    print("### Error adjusting lineup ###")
                    return response

            move_bench_players = [{
                    "playerId": "{}".format(player[0]),
                    "type": "LINEUP",
                    "fromLineupSlotId": "{}".format(20),
                    "toLineupSlotId": "{}".format(player[1])
                } for player in team_adjustments[['id', 'adjusted_slot']].values]
            adjustment["items"] = move_bench_players
            response = requests.post(self.ENDPOINT + "/transactions/", json=adjustment, cookies=self.cookies)
            if response.ok is False:
                print("### Error adjusting lineup and moving players to starters ###")
                return response

            print("<--> {} lineup adjustments made".format(adjustments_needed))
            print("-" * 100)
            if len(current_starters) > 0:
                benched_points = sum(current_starters.projection)
                for player in current_starters[['name', 'current_slot', 'projection']].values:
                    print("--> Benched {} from the {} position (projected: {})".format(player[0], player[1], player[2]))
            else:
                benched_points = 0
            current_bench = team_adjustments[team_adjustments.current_slot_id >= 20]
            for player in current_bench[['name', 'adjusted_slot', 'projection']].values:
                print("<-- Starting {} in the {} position (projected: {})".format(player[0], POSITION_MAP[player[1]], player[2]))
            starting_points = sum(current_bench.projection)
            print("-" * 50)
            print("-" * 50)
            print("Projected for {} more points".format(float(starting_points - benched_points)))
            return {"benched": list(current_starters.name), "starting": list(current_bench.name)}
        else:
            print("<--> No adjustments needed")
            return {"benched": [], "starting": []}

ImportError: No module named 'ff_espn_api'